In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Professional plotting style
plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'axes.grid': False,
    'font.size': 11,
    'axes.labelsize': 12,
    'axes.titlesize': 13,
    'figure.figsize': (12, 5)
})
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
print('Setup complete.')
from statsmodels.stats.diagnostic import acorr_ljungbox

def plot_ljungbox_pvalues(series, noestimatedcoef=0, nolags=25,
                          title='P-values for Ljung-Box Test', figsize=(12, 5)):
    """Plot p-values for the Ljung-Box test at multiple lags."""
    lags = np.arange(1, nolags + 1)
    lb_results = acorr_ljungbox(series, lags=nolags, model_df=noestimatedcoef)
    pvalues = lb_results['lb_pvalue'].values
    fig, ax = plt.subplots(figsize=figsize)
    fig.patch.set_alpha(0); ax.patch.set_alpha(0); ax.grid(False)
    ax.scatter(lags, pvalues, color='steelblue', s=40, zorder=3)
    ax.axhline(y=0.05, color='red', linestyle='--', linewidth=1.0, label='5% significance')
    ax.set_xlabel('Lag'); ax.set_ylabel('P-value')
    ax.set_title(title, fontsize=13, fontweight='bold')
    ax.set_ylim(-0.05, 1.05)
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.13), ncol=1, frameon=False)
    plt.tight_layout(rect=[0, 0.08, 1, 1])
    return fig, ax


## 1. Load Data


In [ ]:
from statsmodels.tsa.regime_switching.tests.test_markov_regression import fedfunds, ogap, inf

index = pd.date_range('1954-07-01', '2010-10-01', freq='QS')
dta_fedfunds = pd.Series(fedfunds, index=index, name='fedfunds')
dta_ogap = pd.Series(ogap, index=index, name='ogap')
dta_inf = pd.Series(inf, index=index, name='inf')

print(f'Observations: {len(dta_fedfunds)}')


In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(14, 9), sharex=True)
fig.patch.set_alpha(0)
for ax, (s, lab, col) in zip(axes, [(dta_fedfunds, 'Fed Funds Rate', 'steelblue'),
        (dta_ogap, 'Output Gap', 'darkorange'), (dta_inf, 'Inflation', 'seagreen')]):
    ax.patch.set_alpha(0); ax.grid(False)
    ax.plot(s, color=col, linewidth=1.0); ax.set_ylabel(lab)
axes[0].set_title('Federal Funds Rate, Output Gap, and Inflation', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.savefig('P07_chart01.png', dpi=150, bbox_inches='tight', transparent=True)
plt.show()


## 2. Model 1 — 2 Regimes, 3 Exogenous


In [ ]:
fedfundslag = dta_fedfunds.shift(1)
exog = pd.concat((fedfundslag, dta_ogap, dta_inf), axis=1).iloc[4:]
endog = dta_fedfunds.iloc[4:]

res1 = sm.tsa.MarkovRegression(endog=endog, k_regimes=2,
    exog=exog, order=0, switching_variance=False).fit()
print(res1.summary())


In [ ]:
fig, ax = plot_ljungbox_pvalues(res1.resid, nolags=25,
    title='Ljung-Box P-values — Model 1 (2 Regimes, 3 Exog)')
plt.savefig('P07_chart02.png', dpi=150, bbox_inches='tight', transparent=True)
plt.show()
print('⚠️  INADEQUATE — most p-values < 0.05.')


## 3. Model 2 — 3 Regimes, 3 Exogenous


In [ ]:
mod2 = sm.tsa.MarkovRegression(endog=endog, k_regimes=3,
    exog=exog, order=0, switching_variance=False)
np.random.seed(12345)
res2 = mod2.fit(search_reps=20)
print(res2.summary())


In [ ]:
fig, ax = plot_ljungbox_pvalues(res2.resid, nolags=25,
    title='Ljung-Box P-values — Model 2 (3 Regimes, 3 Exog)')
plt.savefig('P07_chart03.png', dpi=150, bbox_inches='tight', transparent=True)
plt.show()
print('⚠️  Still inadequate — add second lag of fedfunds.')


## 4. Model 3 — 3 Regimes, 4 Exogenous ✅


In [ ]:
fedfundslag2 = dta_fedfunds.shift(2)
exog_4 = pd.concat((fedfundslag, fedfundslag2, dta_ogap, dta_inf), axis=1).iloc[4:]

mod3 = sm.tsa.MarkovRegression(endog=endog, k_regimes=3,
    exog=exog_4, order=0, switching_variance=False)
np.random.seed(123457)
res3 = mod3.fit(search_reps=20)
print(res3.summary())


In [ ]:
fig, ax = plot_ljungbox_pvalues(res3.resid, nolags=25,
    title='Ljung-Box P-values — Model 3 (3 Regimes, 4 Exog) ✅')
plt.savefig('P07_chart04.png', dpi=150, bbox_inches='tight', transparent=True)
plt.show()
print('✅ All p-values > 0.05 → model is ADEQUATE.')


## 5. Smoothed Regime Probabilities (Final Model)


In [ ]:
smoothed = res3.smoothed_marginal_probabilities
fig, axes = plt.subplots(4, 1, figsize=(14, 12), sharex=True)
fig.patch.set_alpha(0)
colors = ['steelblue', 'darkorange', 'seagreen']

ax = axes[0]; ax.patch.set_alpha(0); ax.grid(False)
ax.plot(endog, color='black', linewidth=0.8); ax.set_ylabel('Fed Funds Rate')
ax.set_title('Federal Funds Rate and Smoothed Regime Probabilities (Final Model)', fontsize=13, fontweight='bold')

for i in range(3):
    ax = axes[i+1]; ax.patch.set_alpha(0); ax.grid(False)
    ax.fill_between(smoothed.index, smoothed.iloc[:, i], alpha=0.6, color=colors[i])
    ax.set_ylabel(f'P(Regime {i})'); ax.set_ylim(-0.05, 1.05)

plt.tight_layout()
plt.savefig('P07_chart05.png', dpi=150, bbox_inches='tight', transparent=True)
plt.show()


## 6. Model Comparison


In [ ]:
comp = pd.DataFrame({
    'Model': ['2 Reg, 3 Exog', '3 Reg, 3 Exog', '3 Reg, 4 Exog'],
    'Log-Lik': [res1.llf, res2.llf, res3.llf],
    'AIC': [res1.aic, res2.aic, res3.aic],
    'BIC': [res1.bic, res2.bic, res3.bic],
    'Adequate': ['No', 'No', 'Yes ✅']
}).set_index('Model')
print(comp.to_string())
